In [2]:
import numpy as np
import sys
from os.path import join

dir_path = "/Users/jeremygoldwasser/Desktop/RankSHAP"
data_path = join(dir_path, "Experiments", "Data")
sys.path.append(join(dir_path, "HelperFiles"))
from helper import *
from shapley_sampling2 import *
from train_models import *
from load_data import *

import warnings
warnings.filterwarnings('ignore')



/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load dataset and train model

In [3]:
dataset = "census"
# X_train, y_train, X_test, y_test, mapping_dict = make_data(data_path, dataset)
# save_data(data_path, dataset, X_train, y_train, X_test, y_test, mapping_dict)
X_train, y_train, X_test, y_test, mapping_dict = load_data(data_path, dataset)
xloc = X_test[1:2]

In [3]:
model = train_neural_net(X_train, y_train)

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))

Class imbalance: 75%
NN 82% accuracy


## Preliminary run of Shapley sampling

Not infrequently, expected number of samples to be significant exceeds max number of perms; sampling it, though, it's significant. This suggests our distribution is too conservative. Maybe we can remove the factor of 2.

In [4]:
# %run shapley_sampling2
np.random.seed(6)

shap_vals, diffs_all_feats = shapley_sampling_adaptive(model, X_train, xloc, K=2, alpha=0.2, mapping_dict=mapping_dict, max_n_perms=10000)

In [5]:
print(np.sum(shap_vals), model(xloc) - np.mean(model(X_train)))

print(np.sort(np.abs(shap_vals))[::-1])

0.4424208630654952 0.53665346
[0.24634868 0.10867932 0.08058421 0.0669929  0.04353731 0.03680945
 0.03307947 0.02164673 0.02095484 0.01365652 0.0108488  0.00482027]


# Simulate Experimental FWER

## Model itself

Recommended number of samples often exceeds max # perms by a LOT, but then running the max # usually works. Makes me question whether the boundary is valid.

Using n_init=30 makes FWER more interesting, so that's what we show. Often n_init=50 makes the FWER 0. 

Still not sure one way or the other whether we need the factor of 2. Without, it's the Welch's t-test statistic. All these results are with the factor of 2.

results in 11/9 table

In [19]:
# %run shapley_sampling2
# np.random.seed(2) # Unusual
K = 5
top_K = []
for i in range(5):
    # print(i)
    shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(model, X_train, xloc, K=K, alpha=0.2, 
                            mapping_dict=mapping_dict, max_n_perms=10000, n_init=100, factor_of_two=True,
                            n_equal=False)
    if shap_vals2 != "NA":
        est_top_K = get_ordering(shap_vals2)[:K]
        top_K.append(est_top_K)
        print(len(top_K))
        # if (i+1) % 5==0: 
        #     print(i+1)

1
2
3
Hit max # perms without converging. Returning `NA`.
4


In [20]:
top_K = np.array(top_K)
most_common_row = mode_rows(top_K)
ct = 0
for idx in range(top_K.shape[0]):
    if np.array_equal(most_common_row, top_K[idx]):
        ct += 1
print(np.round(1 - ct / top_K.shape[0], 2)) # "FWER
print([len(diffs_all_feats2[j]) for j in range(len(diffs_all_feats2))])
print(top_K.shape[0], i)

0.0
[100, 100, 100, 100, 10000, 4937, 10000, 10000, 100, 100, 100, 10000]
4 4


In [7]:
# import pickle
# with open("test", "wb") as fp:
#     pickle.dump(diffs_all_feats2, fp)

# with open("test", "rb") as fp:
#     diffs_all_feats = pickle.load(fp)
# len(diffs_all_feats)

## Quadratic approximation
- GLM, Credit, BRCA, K=5: controls FWER
- NN, Credit, K=2: doesn't converge

In [8]:
# %run shapley_sampling2
# np.random.seed(1)
# K = 5
# true_order = np.argsort(np.abs(true_shap_vals))[::-1]
# true_top_K = true_order[:K]
# same_top_K = []
# for i in range(100):
#     shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(approx, X_train, xloc, K=K,
#                                                 alpha=0.2, mapping_dict=mapping_dict)
#     est_top_K = np.argsort(np.abs(shap_vals2))[::-1][:K]
#     has_same_top_K = np.array_equal(true_top_K, est_top_K)
#     same_top_K.append(has_same_top_K)
#     if (i+1) % 5==0: print(i+1, np.round(1-np.mean(same_top_K),2)) # FWER
# np.round(1-np.mean(same_top_K),2) 

### Can we use CVX? Don't think so.

In [6]:
# Can we use cvx?
import cvxpy as cp
import numpy as np

In [23]:
np.random.seed(1)
Delta = 10
var1 = 1
var2 = 10
b = -np.array([var2, var1])
Z_a = 1.96

A = np.zeros((2,2))
# A[0, 1] = (Delta / 2*Z_a)**2/2 # Must be symmetric
A[0, 1] = 0.5*(Delta / Z_a)**2/2
A[1,0] = 0.5*(Delta / Z_a)**2/2
c = np.ones(2)
# Define and solve the CVXPY problem.
x = cp.Variable(2)
prob = cp.Problem(cp.Minimize(c.T@x),
                 [x.T@A@x + b.T@x == 0, x >= 0])
prob.solve()


DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
QuadForm(var131, [[0.         6.50770512]
 [6.50770512 0.        ]]) + [-10.  -1.] @ var131 == 0.0 , because the following subexpressions are not:
|--  QuadForm(var131, [[0.         6.50770512]
 [6.50770512 0.        ]])

In [9]:
dataset = "census"
X_train, y_train, X_test, y_test, mapping_dict = load_data(join(dir_path, "Experiments", "Data"), dataset)
model = train_neural_net(X_train, y_train)

i = 1
N_runs = 10
print(i)
xloc = X_test[i:(i+1)]
shap_vals_all = []
top_K = []
while len(top_K) < N_runs:
    shap_vals, _ = shapley_sampling_adaptive(model, X_train, xloc, K=K, alpha=0.2, 
                            mapping_dict=mapping_dict, max_n_perms=10000, n_init=100, n_equal=False)
    if isinstance(shap_vals, np.ndarray):
        #### Compute empirical FWER #####
        est_top_K = get_ordering(shap_vals)[:K]
        top_K.append(est_top_K)
        shap_vals_all.append(shap_vals)
        if len(top_K) % 1 == 0: # 20
            top_K_arr = np.array(top_K)
            most_common_row = mode_rows(top_K_arr)
            ct = 0
            for idx in range(len(top_K)):
                if np.array_equal(most_common_row, top_K_arr[idx]):
                    ct += 1
            fwer = 1 - ct / len(top_K)
            print(len(top_K), np.round(fwer, 2)) # "FWER

1
Hit max # perms without converging. Returning `NA`.
1 0.0
2 0.0
Hit max # perms without converging. Returning `NA`.
3 0.0
4 0.0
Hit max # perms without converging. Returning `NA`.
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
Hit max # perms without converging. Returning `NA`.
10 0.0
